### A stock screener that takes as input a list of stock tickers and calculates the realtive strength of each stock against its sector over the last year. It then screens each stock for technical criteria. It outputs a CSV file with the stocks selected by the first screener, marked for technical analysis by the second screener. This resulting CSV file is used as the basis for the next screening process, to over time achieve a continuous overview of stock performance.

In [13]:
# requirements
import talib
import numpy as np
import pandas as pd
pd.core.common.is_list_like = pd.api.types.is_list_like
import pandas_datareader.data as web 
import datetime
import rsa_func
from datetime import timedelta
from talib.abstract import *

In [15]:
# calculates the relative strength of a stock
def calc_rsa_ratio(index, coin):
    rsa_t = (coin["close_price"].iloc[0]/index['close_price'].iloc[0])
    r = 100 / rsa_t
    rsa_r = pd.DataFrame((((coin["close_price"]/index['close_price'])* r))-100)
    return rsa_r

In [16]:
# calculates moving averages of the relative strength index of a stock
def rsa_ma_c(df):
    df['ma50'] = SMA(df, timeperiod = 50, price = ["close_price"])
    df['ma150'] = SMA(df, timeperiod = 150, price = ["close_price"])
    df1 = df[['ma50','ma150']]
    return df1

In [14]:
# calculates percentage values for stock prices
def coin_percent (coin):
    c = 100 / coin["close_price"].iloc[0]
    coin_p = pd.DataFrame((coin["close_price"]* c) - 100)
    return coin_p

In [17]:
# updates the previous output with the newly selected stocks. Old stocks remain in the list for 8 weeks, even if they 
# currently don't fulfill the criteria.
def update_csv(csv_new, csv_old):
    d1 = pd.read_csv(csv_new)
    d2 = pd.read_csv(csv_old)
    
    today = pd.to_datetime('today')
    months = today - timedelta(60)
    
    d1['Date'] = pd.to_datetime(d1['Date'])
    d2['Date'] = pd.to_datetime(d2['Date'])
    
    d2.rename(columns={d2.columns[0]:'Ticker'}, inplace=True)
    
    d1.drop(d1.columns[0], axis=1, inplace = True)
    
    df = d1.merge(d2, how='outer')
    df1 = df.drop_duplicates(subset=["Ticker"], keep='first')
         
    df2 = df1.loc[df1['Date'] > months]
    df3 = df1.loc[df1['Date'] < months]
    
    return df2, df3

In [18]:
# today's date
today = datetime.datetime.today()
today = today.strftime('%Y-%m-%d')
today

'2018-10-21'

In [19]:
# today one year ago
yt = datetime.datetime.today() - timedelta(days=365)
yeartoday = yt.strftime('%Y-%m-%d')
yeartoday

'2017-10-21'

In [32]:
# the first screening process takes a list of stock tickers and calculates their relative strenght (RSA), 
# and moving averages (MA) for the last year. If the RSA and MAs of the stock fulfill certain criteria
# it is appended to the dataframe.
def screen_first(csv, start, end):
    df = pd.read_csv(csv)
    date = pd.to_datetime('today').strftime("%Y, %m, %d")
  
    select_df = pd.DataFrame(index = ['SYM'], columns = ['Ticker', 'Domain', 'Index', '+', 'Date'])
    
    tickers = df['Ticker'].tolist()
    indices = df['Index'].tolist()
    df.set_index('Ticker', inplace = True)
    
    for ticker, index in zip(tickers, indices):
     
        for att in range (3):
            print ('Loading:', ticker, 'and', index)
            try:
                ind = web.DataReader(index, 'robinhood', start, end)      
                stock = web.DataReader(ticker, 'robinhood', start, end) 
                ind['close_price'] = ind['close_price'].apply(pd.to_numeric)
                stock['close_price'] = stock['close_price'].apply(pd.to_numeric)
                
                ind.reset_index(inplace=True)
                ind.set_index([ind.columns[1]], inplace=True)
                
                stock.reset_index(inplace=True)
                stock.set_index([stock.columns[1]], inplace=True)
                
                try:
                    stock['rsa'] = calc_rsa_ratio(ind, stock)
                    stock['ind_p'] = coin_percent(ind)
                    stock['stock_p'] = coin_percent(stock)
                    stock['stock_EMA10'] = EMA(stock, timeperiod = 10, price = ['stock_p'])
                    stock['stock_EMA20'] = EMA(stock, timeperiod = 20, price = ['stock_p'])
                    stock['stock_MA20'] = SMA(stock, timeperiod = 20, price = ['stock_p'])
                    stock['stock_MA50'] = SMA(stock, timeperiod = 50, price = ['stock_p'])
                    stock['stock_MA150'] = SMA(stock, timeperiod = 150, price = ['stock_p'])
                    stock['stock_MA200'] = SMA(stock, timeperiod = 200, price = ['stock_p'])
                    stock['stock_MA250'] = SMA(stock, timeperiod = 250, price = ['stock_p'])
                    stock['rsa_EMA20'] = EMA(stock, timeperiod = 20, price = ['rsa'])
                    stock['rsa_MA20'] = SMA(stock, timeperiod = 20, price = ['rsa'])
                    stock['rsa_MA30'] = SMA(stock, timeperiod = 30, price = ['rsa'])
                    stock['rsa_MA150'] = SMA(stock, timeperiod = 150, price = ['rsa'])
                    stock['rsa_MA200'] = SMA(stock, timeperiod = 200, price = ['rsa'])
                    stock['rsa_MA250'] = SMA(stock, timeperiod = 250, price = ['rsa'])
                except Exception as e:
                    print (e)
                    pass  
                else:
                    if stock['rsa_MA150'].iloc[-1] > stock['rsa_MA200'].iloc[-1] > stock['rsa_MA250'].iloc[-1]\
                        and stock['stock_MA150'].iloc[-1] > stock['stock_MA200'].iloc[-1]\
                        > stock['stock_MA250'].iloc[-1]\
                        and stock['rsa_MA150'].iloc[-1] > 10\
                        and stock['rsa_MA200'].iloc[-1] > 7\
                        and stock['rsa_MA250'].iloc[-1] > 5\
                        and stock['stock_MA150'].iloc[-1] > 20\
                        and stock['stock_MA200'].iloc[-1] > 17\
                        and stock['stock_MA250'].iloc[-1] > 14\
                        and stock['stock_MA50'].iloc[-1] > stock['stock_MA150'].iloc[-1]\
                        and (stock['stock_MA250'].iloc[-1] / stock['stock_MA200'].iloc[-1]) < 0.99\
                        and (stock['stock_MA200'].iloc[-1] / stock['stock_MA150'].iloc[-1]) < 0.98:              
                        select_df.at[ticker, 'Ticker'] = ticker
                        select_df.at[ticker, 'Domain'] = df.at[ticker, 'Domain']
                        select_df.at[ticker, 'Index'] = index
                        select_df.at[ticker, 'Date'] = date
                        select_df.at[ticker, '+'] = 'Uptrend 1.1'
                            
                    if (stock['rsa_EMA20'].iloc[-1] - stock['rsa_EMA20'].iloc[-50]) > 5 \
                    and (stock['stock_EMA20'].iloc[-1] - stock['stock_EMA20'].iloc[-50]) > 13:
                        select_df.at[ticker, 'Ticker'] = ticker
                        select_df.at[ticker, 'Domain'] = df.at[ticker, 'Domain']
                        select_df.at[ticker, 'Index'] = index
                        select_df.at[ticker, 'Date'] = date
                        select_df.at[ticker, '+'] = 'Breakout 1.0'
                    
                        
                    if stock['stock_MA250'].iloc[-1] <= stock['stock_MA150'].iloc[-1] * 1.03\
                    and stock['stock_MA250'].iloc[-1] >= stock['stock_MA150'].iloc[-1]\
                    - (stock['stock_MA150'].iloc[-1] * 0.03)\
                    and stock['stock_MA250'].iloc[-1] <= stock['stock_MA200'].iloc[-1] * 1.02\
                    and stock['stock_MA250'].iloc[-1] >= stock['stock_MA200'].iloc[-1]\
                    - (stock['stock_MA200'].iloc[-1] * 0.02)\
                    and stock['stock_EMA10'].iloc[-1] < stock['stock_MA250'].iloc[-1] * 0.9:
                        select_df.at[ticker, 'Ticker'] = ticker
                        select_df.at[ticker, 'Domain'] = df.at[ticker, 'Domain']
                        select_df.at[ticker, 'Index'] = index
                        select_df.at[ticker, 'Date'] = date
                        select_df.at[ticker, '+'] = 'Dip of the Range'    
                    
            except Exception as e:
                print (ticker, e) 
                pass
            else:
                break
                   

    return select_df

In [ ]:
eins = screen_first('/Users/sophiehammer/bot/google_get/_ALL for RSAr/_ALL csv for RSAr/All the Market CSV - Complete (2).csv', yeartoday, today)

In [22]:
eins.head()

,Ticker,Domain,Index,+,Date
SYM,NaN,NaN,NaN,NaN,NaN
AMZN,AMZN,Cyclicals XLY,XLY,Uptrend 1.1,"2018, 10, 21"
NFLX,NFLX,Cyclicals XLY Large,XLY,Uptrend 1.1,"2018, 10, 21"
NKE,NKE,Cyclicals XLY,XLY,Uptrend 1.1,"2018, 10, 21"
TJX,TJX,Cyclicals XLY,XLY,Breakout 1.0,"2018, 10, 21"


In [23]:
# save output to csv
eins.to_csv('/Users/sophiehammer/bot/google_get/_Weekly Routine/all_list_20181001.csv')

In [24]:
# compare and update the previous output
x, y = update_csv('/Users/sophiehammer/bot/google_get/_Weekly Routine/all_list_20181001.csv', '/Users/sophiehammer/bot/google_get/_Daily Routine/all_screen_20181014.csv')

In [25]:
x.head()

,Ticker,Domain,Index,+,Date,+-1%SMA50,+-2%SMA150,stoch 14-6-6,stoch 5-3-3,RSI 14,RSI 10,MACD,BB,SAR,Engulfing,Y_P,50d_P
1,AMZN,Cyclicals XLY,XLY,Uptrend 1.1,2018-10-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NFLX,Cyclicals XLY Large,XLY,Uptrend 1.1,2018-10-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NKE,Cyclicals XLY,XLY,Uptrend 1.1,2018-10-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TJX,Cyclicals XLY,XLY,Breakout 1.0,2018-10-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,FOXA,Cyclicals XLY,XLY,Uptrend 1.1,2018-10-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
# check for stocks that fell out of the list in this cycle
y

,Ticker,Domain,Index,+,Date,+-1%SMA50,+-2%SMA150,stoch 14-6-6,stoch 5-3-3,RSI 14,RSI 10,MACD,BB,SAR,Engulfing,Y_P,50d_P


In [27]:
# save the new output to csv
x.to_csv('/Users/sophiehammer/bot/google_get/_Weekly Routine/all_update_20181001.csv')

In [33]:
# the second screning process takes the output of the first screener and checks each stock for certain 
# technical analysis critera. It outputs a dataframe where each stock is marked if it fulfills the criteria. 
def screen_second(csv, start, end):
    df = pd.DataFrame(index = ['Ticker'], \
                      columns = ['Domain', 'Index', 'Date', '+-1%SMA50','+-2%SMA150',\
                                 'stoch 14-6-6', 'stoch 5-3-3', 'RSI 14', 'RSI 10', 'MACD'\
                                 , 'BB', 'SAR', 'Engulfing', '+', 'Y_P', '50d_P'])
    inp = pd.read_csv(csv)
    syms = inp['Ticker'].tolist()
    inp.set_index('Ticker', inplace=True)
    
    for sym in syms:
        for att in range(3):
            try:
                print ('Loading:', sym)
                stock = web.DataReader(sym, 'robinhood', start, end) 
                stock['close_price'] = stock['close_price'].apply(pd.to_numeric)
                stock['high_price'] = stock['high_price'].apply(pd.to_numeric)
                stock['low_price'] = stock['low_price'].apply(pd.to_numeric)
                stock['open_price'] = stock['open_price'].apply(pd.to_numeric)
                
                df.loc[sym] = ['','','','','','','','','','','','','','','',''] 
            
                stock['SMA50'] = SMA(stock, timeperiod=50, price=['close_price'])
                stock['stock_MA20'] = SMA(stock, timeperiod=20, price=['close_price'])
                stock['SMA150'] = SMA(stock, timeperiod=150, price=['close_price'])
                stock[['slowk14', 'slowd14']] = STOCH(stock, 14, 6, 0, 6, 0,\
                                                      prices=['high_price', 'low_price', 'close_price'])
                stock[['slowk', 'slowd']] = STOCH(stock, 5, 3, 0, 3, 0,\
                                                  prices=['high_price', 'low_price', 'close_price'])
                stock['RSI14'] = RSI(stock, timeperiod=14, price=['close_price'])
                stock['RSI10'] = RSI(stock, timeperiod=10, price=['close_price'])
                stock[['macd', 'macdsignal', 'macdhist']] = MACD(stock, fastperiod=12, slowperiod=26, signalperiod=9,
                                                                 price=['close_price'])
                stock[['upper', 'middle', 'lower']] = BBANDS(stock, 20, 2, 2, price=['close_price'])
                stock['SAR'] = SAR(stock, acceleration=0.02, maximum=0.2, prices=['high_price', 'low_price'])
                stock['Engulfing'] = CDLENGULFING(stock, prices=['open_price',\
                                                                 'high_price', 'low_price', 'close_price']) 
            
                if stock['macd'].iloc[-1] > 0 and stock['macd'].iloc[-1] > stock['macdsignal'].iloc[-1]:
                    df.at[sym, 'MACD'] = '+x'
                    df.at[sym, 'Domain'] = inp.at[sym, 'Domain']
                    df.at[sym, 'Index'] = inp.at[sym, 'Index']
                    df.at[sym, 'Date'] = inp.at[sym, 'Date']
                    df.at[sym, '+'] = inp.at[sym, '+']
                    df.at[sym, 'Y_P'] = round((stock['close_price'].iloc[-1] - stock['close_price'].iloc[1])\
                                              / stock['close_price'].iloc[1] * 100 , 2)
                    df.at[sym, '50d_P'] = round((stock['close_price'].iloc[-1] - stock['close_price'].iloc[-50])\
                                                / stock['close_price'].iloc[-50] * 100, 2)
                
                if stock['macd'].iloc[-1] < 0 and stock['macd'].iloc[-1] > stock['macdsignal'].iloc[-1]:
                    df.at[sym, 'MACD'] = '-x'
                    df.at[sym, 'Domain'] = inp.at[sym, 'Domain']
                    df.at[sym, 'Index'] = inp.at[sym, 'Index']
                    df.at[sym, 'Date'] = inp.at[sym, 'Date']
                    df.at[sym, '+'] = inp.at[sym, '+']
                    df.at[sym, 'Y_P'] = round((stock['close_price'].iloc[-1] - stock['close_price'].iloc[1])\
                                              / stock['close_price'].iloc[1] * 100 , 2)
                    df.at[sym, '50d_P'] = round((stock['close_price'].iloc[-1] - stock['close_price'].iloc[-50])\
                                                / stock['close_price'].iloc[-50] * 100, 2)
                    
                if stock['macd'].iloc[-1] > 0 and stock['macd'].iloc[-1] < stock['macdsignal'].iloc[-1]:
                    df.at[sym, 'MACD'] = '+o'
                    df.at[sym, 'Domain'] = inp.at[sym, 'Domain']
                    df.at[sym, 'Index'] = inp.at[sym, 'Index']
                    df.at[sym, 'Date'] = inp.at[sym, 'Date']
                    df.at[sym, '+'] = inp.at[sym, '+']
                    df.at[sym, 'Y_P'] = round((stock['close_price'].iloc[-1] - stock['close_price'].iloc[1])\
                                              / stock['close_price'].iloc[1] * 100 , 2)
                    df.at[sym, '50d_P'] = round((stock['close_price'].iloc[-1] - stock['close_price'].iloc[-50])\
                                                / stock['close_price'].iloc[-50] * 100, 2)    
                    
                if stock['macd'].iloc[-1] < 0 and stock['macd'].iloc[-1] < stock['macdsignal'].iloc[-1]:
                    df.at[sym, 'MACD'] = '-o'
                    df.at[sym, 'Domain'] = inp.at[sym, 'Domain']
                    df.at[sym, 'Index'] = inp.at[sym, 'Index']
                    df.at[sym, 'Date'] = inp.at[sym, 'Date']
                    df.at[sym, '+'] = inp.at[sym, '+']
                    df.at[sym, 'Y_P'] = round((stock['close_price'].iloc[-1] - stock['close_price'].iloc[1])\
                                              / stock['close_price'].iloc[1] * 100 , 2)
                    df.at[sym, '50d_P'] = round((stock['close_price'].iloc[-1] - stock['close_price'].iloc[-50])\
                                                / stock['close_price'].iloc[-50] * 100, 2)
                
                if stock['close_price'].iloc[-1] > stock['SAR'].iloc[-1]:
                    df.at[sym, 'SAR'] = 'x'
                
                if stock['close_price'].iloc[-1] < stock['SAR'].iloc[-1]:
                    df.at[sym, 'SAR'] = 'o' 
                
                if stock['close_price'].iloc[-1] <= stock['SMA50'].iloc[-1]\
                * 1.01 and stock['close_price'].iloc[-1] >= stock['SMA50'].iloc[-1]\
                - (stock['SMA50'].iloc[-1] * 0.01):
                    df.at[sym, '+-1%SMA50'] = 'x' 
                
                if stock['close_price'].iloc[-1] <= stock['SMA150'].iloc[-1]\
                * 1.02 and stock['close_price'].iloc[-1] >= stock['SMA150'].iloc[-1]\
                - (stock['SMA50'].iloc[-1] * 0.02):
                    df.at[sym, '+-2%SMA150'] = 'x'    
            
                if stock['slowk'].iloc[-1] < 20:
                    df.at[sym, 'stoch 5-3-3'] = 'x'
                
                if stock['slowk'].iloc[-1] > 80:
                    df.at[sym, 'stoch 5-3-3'] = 'o'    
            
                if stock['slowk14'].iloc[-1] < 20:
                    df.at[sym, 'stoch 14-6-6'] = 'x'
            
                if stock['slowk14'].iloc[-1] > 80:
                    df.at[sym, 'stoch 14-6-6'] = 'o'
                
                if stock['RSI10'].iloc[-1] < 30:
                    df.at[sym, 'RSI 10'] = 'x'
                
                if stock['RSI10'].iloc[-1] > 70:
                    df.at[sym, 'RSI 10'] = 'o'    
            
                if stock['RSI14'].iloc[-1] < 30:
                    df.at[sym, 'RSI 14'] = 'x'
                
                if stock['RSI14'].iloc[-1] > 70:
                    df.at[sym, 'RSI 14'] = 'o'    

                if stock['low_price'].iloc[-1] < stock['lower'].iloc[-1]:
                    df.at[sym, 'BB'] = 'x'
                
                if stock['high_price'].iloc[-1] > stock['upper'].iloc[-1]:
                    df.at[sym, 'BB'] = 'o'     
                
                if stock['Engulfing'].iloc[-1] == 100:
                    df.at[sym, 'Engulfing'] = 'x'
             
                if stock['Engulfing'].iloc[-1] == -100:
                    df.at[sym, 'Engulfing'] = 'o'
            
            except Exception as e:
                print ('Could not load:', sym, e)
                continue         
            else:
                break
            
    return df

In [ ]:
zwei = screen_second('/Users/sophiehammer/bot/google_get/_Weekly Routine/all_update_20181001.csv', yeartoday, today)

In [30]:
zwei.head()

,Domain,Index,Date,+-1%SMA50,+-2%SMA150,stoch 14-6-6,stoch 5-3-3,RSI 14,RSI 10,MACD,BB,SAR,Engulfing,+,Y_P,50d_P
Ticker,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AMZN,Cyclicals XLY,XLY,2018-10-21,,x,,,,,-o,,o,,Uptrend 1.1,82.56,-6.48
NFLX,Cyclicals XLY Large,XLY,2018-10-21,,,,,,,-o,,x,,Uptrend 1.1,72.84,-3.82
NKE,Cyclicals XLY,XLY,2018-10-21,,x,x,,,,-o,,o,,Uptrend 1.1,39.91,-7.85
TJX,Cyclicals XLY,XLY,2018-10-21,x,,,,,,+o,x,o,,Breakout 1.0,51.14,6.94


In [31]:
# save the output to csv
zwei.to_csv('/Users/sophiehammer/bot/google_get/_Daily Routine/all_screen_20181001.csv')